## Scratch pad for initial CAL foundation directory work with Will

In [ ]:
import geopandas as gpd
import os
import pandas as pd
from typing import List, Dict
from pathlib import Path
import matplotlib.pyplot as plt
from utils.data import contains_substring, replace_lots
from utils.io import xls_to_csvs
from utils.gpd import df_to_gdf
from utils.location import city_lat_lon


import warnings
import datetime as dt

warnings.filterwarnings("ignore")
%load_ext jupyter_black

In [ ]:
# combine lists

In [ ]:
# Oil and gas watch data has CO2 sites/pipelines not in the EIA databases

# get ogw data and split it into CSVs:
xls_to_csvs(
    "/Volumes/T5_External/data/oilgas/ogw/ogw_export_20250707.xlsx",
    "/Volumes/T5_External/data/oilgas/ogw",
)

### Define fit-for-purpose functions

In [ ]:
def read_geojson_files(folder_path: str) -> Dict[gpd.GeoDataFrame]:
    """
    Reads all geojson files in the specified folder and returns a dict of GeoDataFrames.

    Args:
        folder_path: The path to the folder containing GeoJSON files.
    Returns:
        list of gdfs from the file
    """
    gdf_dict = dict()

    # Iterate through all files in the specified folder
    for filename in os.listdir(folder_path):
        if filename.endswith(".geojson"):
            file_path = os.path.join(folder_path, filename)
            # Read the GeoJSON file into a GeoDataFrame
            gdf = gpd.read_file(file_path)
            gdf_dict[filename.rsplit("_", 1)[0]] = gdf
            # geo_dataframes.append(gdf)
            print(f"Loaded {filename} with {len(gdf)} features.")

    return gdf_dict

### Get data

In [ ]:
# get the OGW projects data
projects_df = pd.read_csv("/Volumes/T5_External/data/oilgas/ogw/projects.csv")
projects_df.columns = [replace_lots(x.lower()) for x in projects_df.columns.values]
projects_gdf = df_to_gdf(projects_df)
projects_gdf.company.fillna("", inplace=True)

# get the OGW list of UIC permit locations
uic_df = pd.read_csv("/Volumes/T5_External/data/oilgas/ogw/uic_permits.csv")
uic_df.columns = [replace_lots(x.lower()) for x in uic_df.columns.values]
uic_gdf = df_to_gdf(uic_df, lat="well_latitude", lon="well_longitude")
uic_gdf.facility.fillna("", inplace=True)

In [ ]:
# Read in EIA data
gdf_dict = read_geojson_files("/Volumes/T5_External/data/eia")

Loaded CoalMines_US_EIA_3371837949864749161.geojson with 560 features.
Loaded Petroleum_Refineries_US_EIA_-8500459705133754478 1.geojson with 132 features.
Loaded Power_Plants.geojson with 13446 features.
Loaded PetroleumProduct_Terminals_US_EIA_-1807265316411812248.geojson with 1459 features.
Loaded Ethylene_Crackers_US_EIA_-3527313759907648060.geojson with 35 features.
Loaded NaturalGas_InterIntrastate_Pipelines_US_EIA_-6673208179901703463.geojson with 32892 features.
Loaded Hydrocarbon_Gas_Liquids_(HGL)_Market_Hubs.geojson with 3 features.
Loaded HGL_Pipelines_US_EIA_-4268877595463524252.geojson with 133 features.
Loaded NaturalGas_ProcessingPlants_US_EIA_-1067528654751822058.geojson with 478 features.
Loaded Market_Hubs_Natural_Gas_3946026054909568455.geojson with 32 features.
Loaded Lng_ImportExportTerminals_US_EIA_-6607817115306832346.geojson with 9 features.
Loaded Petroleum_Refineries_US_EIA_-8500459705133754478.geojson with 132 features.
Loaded CrudeOil_Pipelines_US_EIA_380455

In [ ]:
# company_list = ["eqt", "equitrans", "rice", "equitable"] # EQT companies/former names

# ExxonMobil company name substrings (including subsidiaries)
company_list = ["exxon", "xto ", "denbury", "pioneer"]

In [ ]:
refinery_gdf = gdf_dict["Petroleum_Refineries_US_EIA"]
[
    x
    for x in list(refinery_gdf.Corp.unique())
    if contains_substring(
        x,
        company_list,
    )
]

[]

In [ ]:
power_gdf = gdf_dict["Power"]
[
    x
    for x in list(power_gdf.Utility_Name.unique())
    if contains_substring(
        x,
        company_list,
    )
]

['Rice University', 'MSC-Rice01 LLC', 'Rice Community Solar Two LLC']

In [ ]:
ng_pipeline_gdf = gdf_dict["NaturalGas_InterIntrastate_Pipelines_US_EIA"]
[
    x
    for x in list(ng_pipeline_gdf.Operator.unique())
    if contains_substring(x, company_list)
]

['Equitrans Inc']

In [ ]:
for i in gdf_dict.keys():
    print(i)

In [ ]:
companies = [
    x for x in uic_df.facility.unique() if contains_substring(x, company_list)
] + [x for x in projects_gdf.company.unique() if contains_substring(x, company_list)]

In [ ]:
names = {
    "Ethylene_Crackers_US_EIA": {
        "company": {"ExxonMobil": ["Exxon/SABIC", "ExxonMobil Chemical Co."]},
        "column_name": "COMPANY",
    },
    "Petroleum_Refineries_US_EIA": {
        "company": {"ExxonMobil": ["EXXON MOBIL CORP"]},
        "column_name": "Corp",
    },
    "Power": {
        "company": {
            "ExxonMobil": [
                "Exxon Mobil Refining and Supply Co.",
                "Exxon Mobil Corp",
                "Exxon Mobil Production Co",
                "ExxonMobil Oil Corp",
                "Denbury Inc",
            ]
        },
        "column_name": "Utility_Name",
    },
    "NaturalGas_ProcessingPlants_US_EIA": {
        "company": {
            "ExxonMobil": [
                "XTO Energy, Inc.",
                "XTO Energy, INC",
                "Exxon",
                "ExxonMobil Corporation",
                "XTO Energy",
            ]
        },
        "column_name": "Owner",
    },
    "CrudeOil_Pipelines_US_EIA": {
        "company": {"ExxonMobil": ["EXXONMOBIL", "EXXONMOBIL WEST COAST"]},
        "column_name": "Opername",
    },
}

In [23]:
for d in list(gdf_dict.keys()):
    print(d)

CoalMines_US_EIA
Petroleum_Refineries_US_EIA
Power
PetroleumProduct_Terminals_US_EIA
Ethylene_Crackers_US_EIA
NaturalGas_InterIntrastate_Pipelines_US_EIA
Hydrocarbon_Gas_Liquids_(HGL)_Market
HGL_Pipelines_US_EIA
NaturalGas_ProcessingPlants_US_EIA
Market_Hubs_Natural_Gas
Lng_ImportExportTerminals_US_EIA
CrudeOil_Pipelines_US_EIA


In [ ]:
gdf_dict["Petroleum_Refineries_US_EIA"]

In [14]:
exxon_list = []
for name, tree in names.items():
    print(name)
    gdf = gdf_dict[name]
    col = tree["column_name"]
    tmp_gdf = gdf[[col, "geometry"]].loc[gdf[col].isin(tree["company"]["ExxonMobil"])]
    tmp_gdf["infrastructure_type"] = name
    exxon_list.append(tmp_gdf)
    del tmp_gdf, gdf

exxon_gdf = gpd.GeoDataFrame(pd.concat(exxon_list), geometry="geometry")

Ethylene_Crackers_US_EIA
Petroleum_Refineries_US_EIA
Power
NaturalGas_ProcessingPlants_US_EIA
CrudeOil_Pipelines_US_EIA


In [15]:
# get US state outlines
DATA_DIR = Path("/Volumes/T5_External/data")
us_gdf = gpd.read_file(DATA_DIR / Path("us_geo/us_states.json"))

In [103]:
xom_pipelines = pipelines_df.loc[pipelines_df.company.isin(pipeline_companies)][
    "pipeline_ogw_id"
].to_list()

In [111]:
sorted(list(pipe_gdf.prjname.unique()))

['134th Street Lateral Project',
 '2021 Auburn A-line Abandonment and Capacity Replacement Project',
 '261 Upgrade Projects',
 'ADCC Pipeline',
 'Adelphia Gateway Project',
 'Aegis Ethane Pipeline',
 'Agua Blanca Pipeline',
 'Air Products CO2 Pipeline',
 'Air Products Hydrogen Lateral',
 'Air Products Natural Gas Pipeline',
 'Alaska Nikiski LNG Pipeline',
 'Apex Pipeline',
 'Appalachia to Market II and Replacement Project',
 'Appalachia to Market Project',
 'Atlantic Bridge Project',
 'Bahia NGL Pipeline',
 'Bailey East Mine Panel 11J Project',
 'Baymark Pipeline',
 'Black Bayou Gas Storage Project',
 'Blackcomb Pipeline',
 'Blackfin Pipeline',
 'Blue Marlin Offshore Port (BMOP) Pipelines',
 'Blue Water Energy Center (BWEC) Pipeline',
 'Bluewater Texas Terminals (BTWX) Pipelines',
 'Borger Express Pipeline',
 'Buckeye Xpress',
 'Buncombe County Enhancement Project',
 'CJ Express Expansion Project',
 'CP Express Pipeline Project',
 'Cameron Extension Project',
 'Cameron Pipeline Expansi

In [112]:
xom_pipe_names = [
    "Denbury Green Pipeline",
    "Denbury Green to CHW Ascension Clean Energy Project",
    "Corpus Christi CO2 Pipeline",
    "Gulf Coast Express Pipeline",
    "Permian Highway Pipeline",
    "Whistler Pipeline and Midland Lateral",
]

In [106]:
pipelines_gdf.to_crs(exxon_gdf.crs, inplace=True)
pipe_gdf.to_crs(exxon_gdf.crs, inplace=True)

In [145]:
univ_df = pd.read_csv(
    "/Users/lindseygulden/dev/REFUSE/outputs/cleaned_university_data.csv"
)
univ_df.columns = [replace_lots(x.lower()) for x in univ_df.columns.values]

In [150]:
sorted(list(univ_df.parent_grantmaker.unique()))

['ADOLPH COORS FOUNDATION',
 'ANNENBERG FOUNDATION',
 'ANSCHUTZ FOUNDATION THE',
 'Acton Institute for The Study of Religion And Liberty',
 'Alabama Power',
 'Alliant Energy Corporation',
 'Ameren Corporation',
 'American Airlines Group Inc.',
 'American Association of Petroleum Geologists',
 'American Electric Power',
 'American Enterprise Institute for Public Policy Research',
 'American Gas Association',
 'American Petroleum Institute',
 'American Public Power Association',
 'Anadarko',
 'Apache',
 'ArcelorMittal',
 'Arch Resources',
 'Association Of American Railroads',
 'Atlas Network',
 'BETH AND RAVENEL CURRY FOUNDATION',
 'BHP',
 'BILL AND MELINDA GATES FOUNDATION',
 'BP',
 'BRADY EDUCATION FOUNDATION, INC.',
 'Berkshire Hathaway Energy',
 'Black Hills Corporation',
 'Boeing Company',
 'CEMEX S.A.B. de C.V.',
 'CMS Energy Corporation',
 'CSX Transportation',
 'Coca-Cola Company',
 'Cummins Inc.',
 'DANIELS FUND',
 'DTE Energy Co',
 'Davis family Foundations',
 'Delta Air Lines,

In [149]:
univ_df[["grant_amount", "parent_grantmaker"]].groupby(
    "parent_grantmaker"
).sum().sort_values(by="grant_amount", ascending=False).head(60)

,grant_amount
parent_grantmaker,
FIDELITY CHARITABLE GIFT FUND,3.127681e+09
LILLY ENDOWMENT INC.,1.892807e+09
BILL AND MELINDA GATES FOUNDATION,1.523711e+09
NATIONAL PHILANTHROPIC TRUST,5.397109e+08
JOHN TEMPLETON FOUNDATION,5.320183e+08
VANGUARD CHARITABLE ENDOWMENT PROGRAM,5.234881e+08
Koch,4.570376e+08
WILLIAM AND FLORA HEWLETT FOUNDATION THE,3.100677e+08
WALTON FAMILY FOUNDATION,2.928186e+08


In [ ]:
xom_univ_df = (
    univ_df[
        [
            "carnegie_naming_cleaned",
            "recipient_city",
            "recipient_state",
            "grant_amount",
        ]
    ]
    .loc[univ_df.parent_grantmaker.isin(["ExxonMobil", "Pioneer Natural Resources"])]
    .groupby(["carnegie_naming_cleaned", "recipient_city", "recipient_state"])
    .sum()
    .reset_index()
)
xom_univ_df.columns = ["university", "city", "state", "usd"]

In [ ]:
for i, row in xom_univ_df.iterrows():
    print(row["university"])
    latlon = city_lat_lon(city=row["city"], state=row["state"])
    xom_univ_df.loc[i, "lat"] = latlon[0]
    xom_univ_df.loc[i, "lon"] = latlon[1]

# convert to a gdf
xom_univ_gdf = df_to_gdf(xom_univ_df, lat="lat", lon="lon")
xom_univ_gdf.to_crs(exxon_gdf.crs)  # get the right crs

Arcadia University
Baylor University
Bowdoin College
Bowie State University
CUNY City College
Carnegie Mellon University
Central State University
City University of New York
Colorado Mesa University
Colorado School of Mines
Colorado State University-Fort Collins
Columbia University in the City of New York
Delaware State University
Duke University
Fairleigh Dickinson University-Metropolitan Campus
Florida Agricultural and Mechanical University
Florida Atlantic University
Florida International University
George Mason University
George Washington University
Georgetown University
Georgia Institute of Technology-Main Campus
Harvard University
Hope College
Howard University
Indiana University-Bloomington
Johns Hopkins University
Lamar Institute of Technology
Lamar University
Lee College
Lindenwood University
Louisiana State University and Agricultural & Mechanical College
Marietta College
Massachusetts Institute of Technology
Morehouse College
New Jersey Institute of Technology
New York Univ

,university,city,state,usd,lat,lon,geometry
0,Arcadia University,Glenside,PA,384000.0,40.103196,-75.151553,POINT (-75.15155 40.10320)
1,Baylor University,Waco,TX,2000.0,31.549190,-97.147463,POINT (-97.14746 31.54919)
2,Bowdoin College,Brunswick,ME,25000.0,43.916653,-69.965944,POINT (-69.96594 43.91665)
3,Bowie State University,Bowie,MD,373840.0,38.942446,-76.730172,POINT (-76.73017 38.94245)
4,CUNY City College,New York City,NY,160000.0,40.712728,-74.006015,POINT (-74.00602 40.71273)
...,...,...,...,...,...,...,...
102,Western Colorado University,Gunnison,CO,1500.0,38.647670,-107.060313,POINT (-107.06031 38.64767)
103,William & Mary,Williamsburg,VA,15000.0,37.270879,-76.707404,POINT (-76.70740 37.27088)
104,Worcester Polytechnic Institute,Worcester,MA,479800.0,42.262562,-71.801888,POINT (-71.80189 42.26256)
105,Xavier University of Louisiana,New Orleans,LA,1533822.0,29.975998,-90.078213,POINT (-90.07821 29.97600)


In [162]:
xom_univ_gdf.sort_values(by="usd", ascending=False).head(60)

,university,city,state,usd,lat,lon,geometry
22,Harvard University,Cambridge,MA,10895000.0,42.365635,-71.104002,POINT (-71.10400 42.36563)
76,University of California-San Francisco,San Francisco,CA,8600000.0,37.779259,-122.419329,POINT (-122.41933 37.77926)
53,Spelman College,Atlanta,GA,3308143.0,33.754466,-84.389815,POINT (-84.38982 33.75447)
33,Massachusetts Institute of Technology,Cambridge,MA,2930000.0,42.365635,-71.104002,POINT (-71.10400 42.36563)
70,Tulane University of Louisiana,New Orleans,LA,2342012.0,29.975998,-90.078213,POINT (-90.07821 29.97600)
26,Johns Hopkins University,Baltimore,MD,1963000.0,39.290882,-76.610759,POINT (-76.61076 39.29088)
83,University of Massachusetts-Amherst,Amherst,MA,1700000.0,42.373195,-72.519876,POINT (-72.51988 42.37319)
105,Xavier University of Louisiana,New Orleans,LA,1533822.0,29.975998,-90.078213,POINT (-90.07821 29.97600)
27,Lamar Institute of Technology,Beaumont,TX,1325213.0,30.082934,-94.098428,POINT (-94.09843 30.08293)
62,The University of Texas Health Science Center ...,Houston,TX,1040000.0,29.758938,-95.367697,POINT (-95.36770 29.75894)


In [14]:
fig, ax = plt.subplots(1, 1, figsize=(16, 9))
us_gdf.plot(ax=ax, facecolor="none", edgecolor="black")
projects_gdf.loc[projects_gdf.company.isin(companies)].plot(
    ax=ax, markersize=10, color="c"
)
uic_gdf.loc[uic_gdf.facility.isin(companies)].plot(
    ax=ax, markersize=15, marker="*", color="k"
)
exxon_gdf.plot(ax=ax, markersize=10, column="infrastructure_type")
pipe_gdf.loc[pipe_gdf.prjname.isin(xom_pipe_names)].plot(ax=ax, color="g")
xom_univ_gdf.plot(ax=ax, marker=".", markersize=6, color="r")

ax.set_xlim(-125, -68)
ax.set_ylim(25, 50)

NameError: name 'plt' is not defined

In [ ]:
crackers_gdf = gdf_dict["Ethylene_Crackers_US_EIA"]
crackers_gdf.COMPANY.unique()

In [ ]:
import requests


def get_naics_code(company_name):
    # Replace spaces with '+' for the API query
    query = company_name.replace(" ", "+")
    url = (
        f"https://api.census.gov/data/2017/naics?get=NAICS,Title&for=us:*&NAICS={query}"
    )

    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise an error for bad responses
        data = response.json()

        # Check if data is returned
        if len(data) > 1:
            # Skip the header row and return the first result
            naics_code, title = data[1]
            return naics_code, title
        else:
            return None, "No NAICS code found for this company."

    except requests.exceptions.RequestException as e:
        return None, f"An error occurred: {e}"


if __name__ == "__main__":
    company_name = input("Enter the company name: ")
    naics_code, title = get_naics_code(company_name)

    if naics_code:
        print(f"NAICS Code: {naics_code}, Title: {title}")
    else:
        print(title)

In [ ]:
hgl_gdf = gdf_dict["Hydrocarbon_Gas_Liquids_(HGL)_Market"]
hgl_gdf

In [ ]:
hgl_pipeline_gdf = gdf_dict["HGL_Pipelines_US_EIA"]
hgl_pipeline_gdf.Opername.unique()

In [ ]:
ng_gdf = gdf_dict["NaturalGas_ProcessingPlants_US_EIA"]
ng_gdf.Owner.fillna("", inplace=True)
[
    x
    for x in list(ng_gdf.Owner.unique())
    if contains_substring(x, ["exxon", "denbury", "xto", "pioneer"])
]

In [71]:
pipe_gdf = gpd.read_file(
    "/Volumes/T5_External/data/oilgas/ogw/ogw_pipeline_files_241121/OGWPipelines_20241121.shp"
)
pipe_gdf.columns = [replace_lots(x.lower()) for x in pipe_gdf.columns.values]

pipelines_df = pd.read_csv("/Volumes/T5_External/data/oilgas/ogw/pipelines.csv")
pipelines_df.columns = [replace_lots(x.lower()) for x in pipelines_df.columns.values]

In [72]:
pipelines_gdf = pipe_gdf.merge(
    pipelines_df, left_on="ogw_id", right_on="pipeline_ogw_id", how="right"
)

In [76]:
pipelines_gdf.company.fillna("", inplace=True)
pipeline_companies = [
    x
    for x in pipelines_gdf.company.unique()
    if contains_substring(x, ["exxon", "mobil", "denbury", "xto", "pioneer"])
]

In [6]:
import os


def play_ding():
    """Plays a ding on a Mac"""
    # TODO make this operating system independent
    os.system("afplay /System/Library/Sounds/Ping.aiff")

In [7]:
play_ding()

In [56]:
pipelines_df.company.fillna("", inplace=True)
sorted(list(pipelines_df.company.unique()))

['',
 'ANR Pipeline Company',
 'Adelphia Gateway LLC',
 'Air Liquide Large Industries US LP',
 'Air Products Blue Energy LLC',
 'Alaska Gasline Development Corporation',
 'Algonquin Gas Transmission, LLC',
 'Algonquin Gas Transmission, LLC, Maritimes & Northeast Pipeline LLC',
 'Alliance Pipeline L.P.',
 'Atlantic Coast Pipeline LLC, Dominion Energy Transmission Inc., Eastern Gas Transmission and Storage, Inc.',
 'Black Bayou Gas Storage, LLC',
 'Black Bear Transmission, LLC',
 'Blue Marlin Offshore Port LLC',
 'Blue Sky Infrastructure',
 'Bluewater Gas Storage LLC',
 'Bridger Pipeline (True Companies)',
 'Cameron Interstate Pipeline, LLC',
 'Capline Pipeline Company, LLC',
 'Carolina Gas Transmission, LLC',
 'Cheniere Corpus Christi Pipeline LP, Corpus Christi Liquefaction LLC',
 'Cheniere Creole Trail Pipeline LP',
 'Cheniere Energy, Inc.',
 'Cheniere Energy, Inc., WhiteWater Midstream, Whistler Pipeline LLC',
 'Chickahominy Power, LLC',
 'City of Clarksville Gas & Water Department',

In [54]:
[
    x
    for x in pipe_gdf.prjname.unique()
    if contains_substring(x, ["exxon", "denbury", "xto", "pioneer", "xom", "mobil"])
]

['Denbury Green Pipeline',
 'Denbury Green to CHW Ascension Clean Energy Project']

In [ ]:
coal_gdf = gdf_dict["CoalMines_US_EIA"]sorted()

In [ ]:
power_gdf = gdf_dict["Power"]
print(len(power_gdf))
power_gdf = power_gdf.loc[
    (power_gdf.Coal_MW > 0) | (power_gdf.NG_MW > 0) | (power_gdf.Crude_MW > 0)
]

In [ ]:
power_gdf["public_utility"] = [
    1
    if contains_substring(
        x,
        [
            "City of",
            "Municipal",
            "Cooperative",
            "Co-op",
            "Coop",
            "Public",
            "Power Authority",
        ],
    )
    else 0
    for x in power_gdf["Utility_Name"]
]

In [ ]:
ng_hub_gdf = gdf_dict["Market_Hubs_Natural_Gas"]

In [ ]:
lng_gdf = gdf_dict["Lng_ImportExportTerminals_US_EIA"]
[x for x in lng_gdf.Owner.unique() if contains_substring(x, company_list)]

In [ ]:
crude_gdf = gdf_dict["CrudeOil_Pipelines_US_EIA"]
[x for x in crude_gdf.Opername.unique() if contains_substring(x, company_list)]

In [13]:
mylist = [1, 2, 3, 4]
mylist.extend([6, 7, 8])
mylist

[1, 2, 3, 4, 6, 7, 8]